In [1]:
import os
import json
from s2ag_corpus.sql import INSERT_PAPER_SQL, CREATE_PAPERS_TABLE_WITH_KEYS


from dotenv import load_dotenv
load_dotenv()
base_dir = os.getenv("BASE_DIR")

In [2]:
from s2ag_corpus.database_catalogue import local_connection

connection = local_connection()

# def delete_papers_from_test_db():
#     with connection.cursor() as cursor:
#         cursor.execute('DELETE FROM papers')
#         connection.commit()

def drop_and_replace_papers_table():
    with connection.cursor() as cursor:
        cursor.execute('drop table if exists papers')
        cursor.execute(CREATE_PAPERS_TABLE_WITH_KEYS)

In [6]:
drop_and_replace_papers_table()
    

# I'm going to delete the rows and time an insertion of 10000 papers

In [0]:
connection.rollback()

In [7]:
test_file = base_dir+'/2024-04-02/papers/first10000papers'
with open(test_file) as f:
    with connection.cursor() as cursor:
        for line in f:
            line = line.strip()
            lj = json.loads(line)
            corpus_id = lj['corpusid']
            cursor.execute(INSERT_PAPER_SQL, (corpus_id, line))
            connection.commit()
    

In [ ]:
delete_papers_from_test_db()

In [ ]:
with open(base_dir+'/papers/first10000papers') as f:
    with connection.cursor() as cursor:
        for line in f:
            line = line.strip()
            lj = json.loads(line)
            corpus_id = lj['corpusid']
            cursor.execute(INSERT_PAPER_SQL, (corpus_id, line))
    connection.commit()

In [ ]:
delete_papers_from_test_db()

In [ ]:
from s2ag_corpus.database_catalogue import CorpusDatabaseCatalogue
catalogue = CorpusDatabaseCatalogue(local_connection())

In [ ]:
with open(base_dir + '/papers/first10000papers') as f:
    jason_dictionaries = [(line, json.loads(line)) for line in f.readlines()]
    records = [(jd['corpusid'], line) for  line, jd in jason_dictionaries]
catalogue.upsert(INSERT_PAPER_SQL, records)

In [ ]:
import csv

In [ ]:
with open(base_dir + '/papers/first10000papers') as f:
    jason_dictionaries = [(line, json.loads(line)) for line in f.readlines()]
    records = [(jd['corpusid'], line.strip()) for  line, jd in jason_dictionaries]
    with open(base_dir+'/first10000papers.csv','w') as csvf:
        writer = csv.writer(csvf, delimiter=',', quoting=csv.QUOTE_NONE, escapechar='\\')
        for record in records:
            writer.writerow(record)

In [ ]:
delete_papers_from_test_db()

In [ ]:
with open(base_dir+'/papers/first10000papers.csv','r') as csvf:
    with connection.cursor() as cursor:
        cursor.copy_from(csvf, 'papers', sep=',', null='')
    connection.commit()

In [ ]:
connection.rollback()